Machine learning para clasificar
Python es el lenguaje preferido para la ciencia de datos debido a NumPy, Pandas y matplotlib, que son herramientas que facilitan el  trabajo con matrices y dibujan gráficos y pueden trabajar con grandes matrices de datos de manera eficiente. Pero Spark está diseñado para trabajar con una enorme cantidad de datos, distribuidos en un clúster.
Datos del paciente del corazón
Encontrará adjunto la base de datos “heart.csv”.-
Las columnas son:
1.  Años
2.  Sexo
3.  Tipo de dolor torácico (4 valores)
4.  Presión arterial en reposo
5.  Colesterol sérico en mg/dl
6.  Azúcar en sangre en ayunas > 120 mg/dl
7.  Resultados electrocardiográficos en reposo (valores 0,1,2)
8.  Frecuencia cardíaca máxima alcanzadaActividad 3
9.  Angina inducida por el ejercicio
10. Oldpeak = depresión del ST inducida por el ejercicio en relación con el reposo
11. Pendiente del segmento ST de ejercicio máximo
12. Número de vasos principales (0-3) coloreados por fluoroscopia
13. Thal: 3 = normal; 6 = defecto fijo; 7 = defecto reversible
El campo que indica si el paciente tiene un problema cardíaco. Los números son los siguientes:

In [ ]:
#!pip install pyspark

In [17]:
#importamos la librerias
import pandas as pd
import numpy as np
from pyspark.sql.functions import lit
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession, DataFrame
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.functions import when
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType



In [18]:
#cargamos los datos
ruta_heart = './datos/heart.csv'

#leemos los datos
datos = pd.read_csv(ruta_heart, delimiter=';')

#mostramos los datos
print(datos.head())

   years  sexo  dolor   pres    col  sug  electro  fcarmax  angina  oldpeak  \
0   70.0   1.0    4.0  130.0  322.0  0.0      2.0    109.0     0.0      2.4   
1   67.0   0.0    3.0  115.0  564.0  0.0      2.0    160.0     0.0      1.6   
2   57.0   1.0    2.0  124.0  261.0  0.0      0.0    141.0     0.0      0.3   
3   64.0   1.0    4.0  128.0  263.0  0.0      0.0    105.0     1.0      0.2   
4   74.0   0.0    2.0  120.0  269.0  0.0      2.0    121.0     1.0      0.2   

   pendst  vasprin  thal  enfermo  
0     2.0      3.0   3.0        1  
1     2.0      0.0   7.0        0  
2     1.0      0.0   7.0        1  
3     2.0      1.0   7.0        0  
4     1.0      1.0   3.0        0  


Entonces, escriba esta función ENFERMO() para marcar 0 como negativo y 1 como  positivo,  porque  la  regresión  logística  binaria  requiere  uno  de  dos resultados

In [20]:
# Define la función personalizada ENFERMO para valores double
def ENFERMO(value):
    # Tolerancia para la comparación de punto flotante
    tolerancia = 0.0001

    # Compara el valor con 3.0 y devuelve 1 si está dentro de la tolerancia, 0 en caso contrario
    if abs(value - 3.0) < tolerancia:
        return 1
    else:
        return 0

# Registra la función como una UDF en PySpark
udf_enfermo = udf(ENFERMO, IntegerType())



In [21]:
datos['thal'] = datos['thal'].apply(ENFERMO)

También  debe crear  el  dataframe  de  Spark  raw_data usando  la  operación transform() y seleccionando solo la columna de características.

In [23]:
# Iniciar una SparkSession
spark = SparkSession.builder.appName("Ejercicio").getOrCreate()

# Cargar el archivo heart.csv en un DataFrame
raw_data = spark.read.csv('./datos/heart.csv', inferSchema=True, header=True, sep=';')

def drop_enfermo(datos: DataFrame) -> DataFrame:
    return datos.drop('enfermo')

# Aplicar la función a tu DataFrame raw_data
raw_data = raw_data.transform(drop_enfermo)
raw_data.show()

# mostrar el total de filas
print(raw_data.count())

+-----+----+-----+-----+-----+---+-------+-------+------+-------+------+-------+----+
|years|sexo|dolor| pres|  col|sug|electro|fcarmax|angina|oldpeak|pendst|vasprin|thal|
+-----+----+-----+-----+-----+---+-------+-------+------+-------+------+-------+----+
| 70.0| 1.0|  4.0|130.0|322.0|0.0|    2.0|  109.0|   0.0|    2.4|   2.0|    3.0| 3.0|
| 67.0| 0.0|  3.0|115.0|564.0|0.0|    2.0|  160.0|   0.0|    1.6|   2.0|    0.0| 7.0|
| 57.0| 1.0|  2.0|124.0|261.0|0.0|    0.0|  141.0|   0.0|    0.3|   1.0|    0.0| 7.0|
| 64.0| 1.0|  4.0|128.0|263.0|0.0|    0.0|  105.0|   1.0|    0.2|   2.0|    1.0| 7.0|
| 74.0| 0.0|  2.0|120.0|269.0|0.0|    2.0|  121.0|   1.0|    0.2|   1.0|    1.0| 3.0|
| 65.0| 1.0|  4.0|120.0|177.0|0.0|    0.0|  140.0|   0.0|    0.4|   1.0|    0.0| 7.0|
| 56.0| 1.0|  3.0|130.0|256.0|1.0|    2.0|  142.0|   1.0|    0.6|   2.0|    1.0| 6.0|
| 59.0| 1.0|  4.0|110.0|239.0|0.0|    2.0|  142.0|   1.0|    1.2|   2.0|    1.0| 7.0|
| 60.0| 1.0|  4.0|140.0|293.0|0.0|    2.0|  170.0|   0

1. Use Standard Scaler para poner todos los números en la misma escala. Esto toma la observación y resta la media, y luego la divide por la desviación estándar.
2. Imprima el estado actual de sus datos

In [24]:
# Crear un VectorAssembler para convertir las columnas de características en un vector
# Reemplaza ['col1', 'col2', 'col3'] con las columnas que deseas escalar
assembler = VectorAssembler(inputCols=['years', 'sexo', 'dolor', 'pres', 'col', 'sug', 'electro', 'fcarmax', 'angina', 'oldpeak', 'pendst', 'vasprin', 'thal'], outputCol='features')

# Transformar el DataFrame
raw_data = assembler.transform(raw_data)

# Crear un StandardScaler
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

# Ajustar el StandardScaler al DataFrame
scaler_model = scaler.fit(raw_data)

# Transformar el DataFrame
raw_data = scaler_model.transform(raw_data)

# Mostrar el DataFrame
raw_data.show()


+-----+----+-----+-----+-----+---+-------+-------+------+-------+------+-------+----+--------------------+--------------------+
|years|sexo|dolor| pres|  col|sug|electro|fcarmax|angina|oldpeak|pendst|vasprin|thal|            features|      scaledFeatures|
+-----+----+-----+-----+-----+---+-------+-------+------+-------+------+-------+----+--------------------+--------------------+
| 70.0| 1.0|  4.0|130.0|322.0|0.0|    2.0|  109.0|   0.0|    2.4|   2.0|    3.0| 3.0|[70.0,1.0,4.0,130...|[7.68465131046640...|
| 67.0| 0.0|  3.0|115.0|564.0|0.0|    2.0|  160.0|   0.0|    1.6|   2.0|    0.0| 7.0|[67.0,0.0,3.0,115...|[7.35530911144641...|
| 57.0| 1.0|  2.0|124.0|261.0|0.0|    0.0|  141.0|   0.0|    0.3|   1.0|    0.0| 7.0|[57.0,1.0,2.0,124...|[6.25750178137978...|
| 64.0| 1.0|  4.0|128.0|263.0|0.0|    0.0|  105.0|   1.0|    0.2|   2.0|    1.0| 7.0|[64.0,1.0,4.0,128...|[7.02596691242642...|
| 74.0| 0.0|  2.0|120.0|269.0|0.0|    2.0|  121.0|   1.0|    0.2|   1.0|    1.0| 3.0|[74.0,0.0,2.0,120..

3. Divida los datos en conjuntos de datos de entrenamiento y prueba. Use 50/50.

In [25]:
# Dividir los datos en conjuntos de entrenamiento y prueba
train_data, test_data = raw_data.randomSplit([0.5, 0.5])

# Mostrar los conjuntos de entrenamiento y prueba
train_data.show()
test_data.show()

+-----+----+-----+-----+-----+---+-------+-------+------+-------+------+-------+----+--------------------+--------------------+
|years|sexo|dolor| pres|  col|sug|electro|fcarmax|angina|oldpeak|pendst|vasprin|thal|            features|      scaledFeatures|
+-----+----+-----+-----+-----+---+-------+-------+------+-------+------+-------+----+--------------------+--------------------+
| 29.0| 1.0|  2.0|130.0|204.0|0.0|    2.0|  202.0|   0.0|    0.0|   1.0|    0.0| 3.0|[29.0,1.0,2.0,130...|[3.18364125719322...|
| 34.0| 1.0|  1.0|118.0|182.0|0.0|    2.0|  174.0|   0.0|    0.0|   1.0|    0.0| 3.0|[34.0,1.0,1.0,118...|[3.73254492222653...|
| 35.0| 0.0|  4.0|138.0|183.0|0.0|    0.0|  182.0|   0.0|    1.4|   1.0|    0.0| 3.0|[35.0,0.0,4.0,138...|[3.84232565523320...|
| 35.0| 1.0|  4.0|120.0|198.0|0.0|    0.0|  130.0|   1.0|    1.6|   2.0|    0.0| 7.0|[35.0,1.0,4.0,120...|[3.84232565523320...|
| 35.0| 1.0|  4.0|126.0|282.0|0.0|    2.0|  156.0|   1.0|    0.0|   1.0|    0.0| 7.0|[35.0,1.0,4.0,126..

4. Cree un modelo de regresión logística y entrénelo

In [26]:
# Crea una instancia de LogisticRegression
lr = LogisticRegression(featuresCol='scaledFeatures', labelCol='label')

# Crea la columna 'label' a partir de la columna 'thal' en los datos de entrenamiento
train_data = train_data.withColumn('label', when(train_data['thal'] == 3, 1).otherwise(0))

# Crea la columna 'label' a partir de la columna 'thal' en los datos de prueba
test_data = test_data.withColumn('label', when(test_data['thal'] == 3, 1).otherwise(0))

# Entrena el modelo
lr_model = lr.fit(train_data)

# Realiza las predicciones
predictions = lr_model.transform(test_data)

# Ahora puedes seleccionar las columnas 'label', 'prediction' y 'probability'
predictions.show()

#mostaral el total de filas
print('Total de filas:', predictions.count())

+-----+----+-----+-----+-----+---+-------+-------+------+-------+------+-------+----+--------------------+--------------------+-----+--------------------+--------------------+----------+
|years|sexo|dolor| pres|  col|sug|electro|fcarmax|angina|oldpeak|pendst|vasprin|thal|            features|      scaledFeatures|label|       rawPrediction|         probability|prediction|
+-----+----+-----+-----+-----+---+-------+-------+------+-------+------+-------+----+--------------------+--------------------+-----+--------------------+--------------------+----------+
| 34.0| 0.0|  2.0|118.0|210.0|0.0|    0.0|  192.0|   0.0|    0.7|   1.0|    0.0| 3.0|[34.0,0.0,2.0,118...|[3.73254492222653...|    1|[-28.884967401239...|[2.85376395150550...|       1.0|
| 38.0| 1.0|  1.0|120.0|231.0|0.0|    0.0|  182.0|   1.0|    3.8|   2.0|    0.0| 7.0|[38.0,1.0,1.0,120...|[4.17166785425319...|    0|[21.9389972273207...|[0.99999999970350...|       0.0|
| 40.0| 1.0|  4.0|110.0|167.0|0.0|    2.0|  114.0|   1.0|    2.0|

5. Finalmente Use la función F de Spark SQL para crear una nueva columna correcta cuando ENFERMO() es igual a la predicción, lo que significa que el resultado predicho es igual a los resultados reales.

In [27]:
# Crea una UDF (función definida por el usuario) a partir de tu función ENFERMO()
udf_enfermo = F.udf(ENFERMO, IntegerType())

# Crea una nueva columna 'enfermo' que aplica la UDF a la columna 'thal'
predictions = predictions.withColumn('enfermo', udf_enfermo(predictions['thal']))

# Crea una nueva columna 'correcto' que es 'true' cuando 'real' es igual a 'prediction' y 'false' en caso contrario
predictions = predictions.withColumn('correcto', F.when(F.col('enfermo') == F.col('prediction'), True).otherwise(False))

# Muestra el DataFrame con la nueva columna
predictions.show()

+-----+----+-----+-----+-----+---+-------+-------+------+-------+------+-------+----+--------------------+--------------------+-----+--------------------+--------------------+----------+-------+--------+
|years|sexo|dolor| pres|  col|sug|electro|fcarmax|angina|oldpeak|pendst|vasprin|thal|            features|      scaledFeatures|label|       rawPrediction|         probability|prediction|enfermo|correcto|
+-----+----+-----+-----+-----+---+-------+-------+------+-------+------+-------+----+--------------------+--------------------+-----+--------------------+--------------------+----------+-------+--------+
| 34.0| 0.0|  2.0|118.0|210.0|0.0|    0.0|  192.0|   0.0|    0.7|   1.0|    0.0| 3.0|[34.0,0.0,2.0,118...|[3.73254492222653...|    1|[-28.884967401239...|[2.85376395150550...|       1.0|      1|    true|
| 38.0| 1.0|  1.0|120.0|231.0|0.0|    0.0|  182.0|   1.0|    3.8|   2.0|    0.0| 7.0|[38.0,1.0,1.0,120...|[4.17166785425319...|    0|[21.9389972273207...|[0.99999999970350...|       0.